In [1]:
# Logistic Regression Classification - Grid search

#Binary Classification
#Sigmiod Function (0-1) having threshold value at 0.5 so clssification done if above 0.5 =class1,below 0.5 =class 0

# drop unwaned column in this datset User ID column is not required so drop User ID, axis =1

# axis=0 mentioning row, Axis =1 mentions Column

# Confusion Matrix
# check type I and type II errors and Type I error should be minimum,

# Model report, ROC curve

# Evalution Matrics= Accuracy, Specificity, Sensitivity, Precision, Recall, f1 score, Macro Average, Weighted Average
# report analysis says all the values are low, so this algorithm is not fit for this problem classification

In [2]:
# importing libraries

import pandas as pd
import numpy as ny
import matplotlib.pyplot as plt


In [3]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [8]:
dataset["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [9]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset[["Purchased"]]

In [10]:
indep.shape

(400, 3)

In [11]:
dep

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [12]:
# Spliting into training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(indep,dep,test_size=1/3,random_state=0)

In [24]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [25]:
from sklearn.linear_model import LogisticRegression

In [30]:
from sklearn.model_selection import GridSearchCV

param_grid = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'],
             'penalty':['l2']} 

grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, Y_train)


C:\Users\user\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:    5.1s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    5.4s finished
C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_weighted', verbose=3)

In [45]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, grid_predictions)

# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(Y_test, grid_predictions)


In [46]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,grid_predictions)
# print classification report


In [47]:
from sklearn.metrics import classification_report
clf_report=classification_report(Y_test,grid_predictions)

In [48]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.89      0.93      0.91        85
           1       0.87      0.80      0.83        49

    accuracy                           0.88       134
   macro avg       0.88      0.86      0.87       134
weighted avg       0.88      0.88      0.88       134



In [49]:
from sklearn.metrics import roc_auc_score

roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

0.9507803121248499

In [50]:
table=pd.DataFrame.from_dict(re)

In [51]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.031216,0.015299,0.003794,0.000612,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.781933,0.729564,0.905338,0.805434,0.073438,2
1,0.025039,0.011182,0.003692,0.001659,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.781933,0.729564,0.905338,0.805434,0.073438,2
2,0.015502,0.005598,0.003677,0.001674,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.794573,0.729564,0.917693,0.813798,0.077739,1
3,0.015232,0.002837,0.003740,0.001060,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.781933,0.729564,0.905338,0.805434,0.073438,2


NameError: name 'matplotlib' is not defined